In [5]:
#조건1 양옆10%만 적용한 것의 학습 (aug2)
import os
import cv2
import json
import glob
from ultralytics import YOLO


BASE_AUGMENTED_PATH = "C:/Users/jenni/Desktop/final_data2" # 증강된 데이터의 최상위 경로 (poisson_harmonized 폴더가 이 아래에 있음)
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_RESULTS_FINAL2" # YOLO 학습 결과가 저장될 경로


def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    """
    Labelme JSON 파일들을 YOLO segmentation .txt 파일로 변환합니다.
    Args:
        json_dir (str): Labelme JSON 파일들이 있는 디렉토리 경로.
        output_txt_dir (str): 변환된 YOLO .txt 파일들을 저장할 디렉토리 경로.
        img_w (int): 원본 이미지의 너비.
        img_h (int): 원본 이미지의 높이.
        label_to_id_map (dict): 클래스 이름(label)을 ID(숫자)로 매핑하는 딕셔너리.
    """
    os.makedirs(output_txt_dir, exist_ok=True)
    json_files = glob.glob(os.path.join(json_dir, "*.json"))

    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt_filename = os.path.basename(json_path).replace(".json", ".txt")
        txt_path = os.path.join(output_txt_dir, txt_filename)

        yolo_lines = []
        for shape in data['shapes']:
            label_name = shape['label']
            if label_name in label_to_id_map:
                class_id = label_to_id_map[label_name]
                points = shape['points']
                
                normalized_points = []
                for x, y in points:
                    # 좌표를 이미지 크기에 맞춰 정규화
                    normalized_points.append(f"{(x / img_w):.6f}")
                    normalized_points.append(f"{(y / img_h):.6f}")
                
                yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
            else:
                print(f"Warning: Class '{label_name}' not found in label_to_id_map. Skipping shape in {json_path}")

        if yolo_lines:
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(yolo_lines))
        # else:
        #    # 객체가 없는 이미지도 빈 .txt 파일로 저장하는 것이 YOLO 학습에 더 안전합니다.
        #    with open(txt_path, 'w') as f:
        #        pass # 빈 파일 생성


def create_data_yaml(dataset_base_path, yaml_output_path, class_names):
    """
    YOLOv8 학습을 위한 data.yaml 파일을 생성합니다.
    Args:
        dataset_base_path (str): 증강된 이미지와 라벨이 있는 상위 디렉토리 (예: 'C:/Users/USER/Desktop/augmented_data1/poisson_harmonized')
        yaml_output_path (str): 생성될 data.yaml 파일의 전체 경로.
        class_names (list): 데이터셋의 모든 클래스 이름 목록.
    """
    images_path = os.path.join(dataset_base_path, "images")
    labels_path = os.path.join(dataset_base_path, "labels_yolo") # YOLO 형식의 라벨 폴더

    # 경로를 상대 경로가 아닌 절대 경로로 지정하여 혼란 방지
    train_img_path = images_path
    val_img_path = images_path # 검증 데이터가 따로 없으면 train과 동일하게 설정

    nc = len(class_names)
    names = class_names

    yaml_content = f"""
path: {dataset_base_path}
train: {os.path.relpath(train_img_path, dataset_base_path)} # path 기준으로 상대 경로
val: {os.path.relpath(val_img_path, dataset_base_path)}

nc: {nc}
names: {names}
"""
    # 실제 YOLO 라벨 경로를 data.yaml이 참조하도록 labels 폴더의 이름을 변경하거나 (labels -> labels_yolo),
    # data.yaml 내부에서 라벨 경로를 명시적으로 지정해야 합니다.
    # 일반적으로 YOLO는 'images' 폴더와 같은 레벨의 'labels' 폴더를 기대합니다.
    # 그래서 'labels_yolo'라는 폴더명을 가정했습니다.

    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' file created successfully.")


def train_yolo_for_method(method, epochs=100):
    """
    특정 증강 방식에 대해 YOLOv8 모델을 학습시키고, 검증 결과를 도출합니다.
    Args:
        method (str): 증강 방식 이름 (예: 'poisson_harmonized').
        epochs (int): 학습할 에포크 수.
    """
    print(f"\n--- Starting YOLOv8 training and validation for method: {method} ---")

    # 1. 각 증강 방식에 맞는 데이터셋 경로 설정
    dataset_path = os.path.join(BASE_AUGMENTED_PATH, method)
    images_path = os.path.join(dataset_path, "images")
    labels_json_path = os.path.join(dataset_path, "labels") # Labelme JSON 라벨이 있는 곳
    labels_yolo_path = os.path.join(dataset_path, "labels_yolo") # YOLO .txt 라벨을 저장할 곳

    # 2. 클래스 정보를 미리 정의
    label2id = {}
    id2label = {}
    try:
        with open("classes.txt", "r", encoding="utf-8") as f:
            class_names_from_file = [line.strip() for line in f if line.strip()]
        for i, name in enumerate(class_names_from_file):
            label2id[name] = i
            id2label[i] = name
        print(f"Loaded classes: {id2label}")
    except FileNotFoundError:
        print("Error: 'classes.txt' not found. Please ensure it's in the same directory or provide class info manually.")
        return
    except Exception as e:
        print(f"Error loading classes.txt: {e}")
        return

    # 이미지 크기를 추정하기 위해 첫 번째 이미지 파일을 읽습니다.
    first_image_path = None
    image_files = glob.glob(os.path.join(images_path, "*.jpg")) + glob.glob(os.path.join(images_path, "*.png"))
    if image_files:
        first_image_path = image_files[0]
        img_temp = cv2.imread(first_image_path)
        if img_temp is None:
            print(f"Error: Could not read image {first_image_path}. Check image file integrity.")
            return
        img_h, img_w = img_temp.shape[:2]
        print(f"Detected image dimensions: W={img_w}, H={img_h}")
    else:
        print(f"Error: No images found in {images_path}. Cannot determine image dimensions for YOLO conversion.")
        return

    # 3. Labelme JSON 라벨을 YOLO Segmentation .txt 형식으로 변환
    print(f"Converting Labelme JSON to YOLO segmentation .txt for {method}...")
    convert_labelme_to_yolo_segmentation(labels_json_path, labels_yolo_path, img_w, img_h, label2id)
    print("Conversion complete.")

    # 4. data.yaml 파일 생성
    yaml_path = os.path.join(dataset_path, "data.yaml")
    create_data_yaml(dataset_path, yaml_path, list(id2label.values()))

    # 5. 항상 동일한 사전 학습 모델로부터 학습 시작
    model = YOLO("yolov8n-seg.pt")
    print(f"Loaded YOLOv8n-seg model.")

    # 6. 모든 학습에 동일한 하이퍼파라미터 적용
    try:
        # 학습 수행
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_{method}",
            project=PROJECT_OUTPUT_PATH, # 결과 저장 최상위 경로
            save=True,
            resume=False
        )
        print(f"--- Training for method '{method}' completed. Results saved to {PROJECT_OUTPUT_PATH}/yolov8n_{method} ---")

        # 7. 학습 완료 후 검증 수행 (자동으로 수행되지만 명시적으로 호출 가능)
        # train 시에 이미 val이 포함되므로, 별도로 model.val()을 호출하지 않아도 결과는 results 객체에 포함됩니다.
        # 하지만 특정 체크포인트를 사용하여 검증하고 싶다면 model.val()을 사용할 수 있습니다.
        # 여기서는 학습 결과 객체에서 직접 metrics를 추출하는 방법을 보여줍니다.
        
        # results 객체에서 metrics 추출
        # YOLOv8의 results 객체는 학습 결과 및 검증 지표를 포함합니다.
        # 정확한 키 이름은 YOLOv8 버전에 따라 약간 다를 수 있으니,
        # results 객체의 내용을 확인하는 것이 좋습니다 (예: print(results.metrics)).
        # 일반적으로 segment 모델의 경우, 다음과 같은 키를 가집니다:
        # metrics['metrics/precision(B)'] (Precision)
        # metrics['metrics/recall(B)'] (Recall)
        # metrics['metrics/mAP50(B)'] (mAP50)
        # metrics['metrics/mAP50-95(B)'] (mAP50-95)
        
        if hasattr(results, 'metrics') and results.metrics:
            print("\n--- Validation Metrics ---")
            
            # Segmentation 모델의 경우 mAP는 'mAP50', 'mAP50-95' 형태로 접근합니다.
            # Precision, Recall, F1-score는 classification 또는 detection에서 더 흔하게 사용되지만,
            # segment 결과에서도 'metrics/precision(B)', 'metrics/recall(B)'와 같은 키로 접근 가능합니다.
            # F1-score는 Precision과 Recall로부터 계산해야 합니다.
            
            """precision = results.metrics.get('metrics/precision(B)', 'N/A')
            recall = results.metrics.get('metrics/recall(B)', 'N/A')
            mAP50 = results.metrics.get('metrics/mAP50(B)', 'N/A')
            mAP = results.metrics.get('metrics/mAP50-95(B)', 'N/A') # mAP (0.5 to 0.95 IOU)

            # F1-score 계산
            f1_score = 'N/A'
            if precision != 'N/A' and recall != 'N/A' and (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)

            print(f"Precision: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
            print(f"F1-Score: {f1_score:.4f}")
            print(f"mAP50: {mAP50:.4f}")
            print(f"mAP (50-95): {mAP:.4f}")
            print("------------------------")"""
                        # 값이 없는 경우 None을 반환하도록 get() 메서드를 사용합니다.
            precision = results.metrics.get('metrics/precision(B)')
            recall = results.metrics.get('metrics/recall(B)')
            mAP50 = results.metrics.get('metrics/mAP50(B)')
            mAP = results.metrics.get('metrics/mAP50-95(B)')
            
            # [추가된 부분] 정확도(top-1 accuracy)를 가져옵니다.
            # 만약 사용하는 모델이 다른 키를 사용한다면 ('metrics/accuracy' 등) 이 부분을 수정해주세요.
            accuracy = results.metrics.get('metrics/accuracy_top1')

            # --- F1-score 계산 ---
            f1_score = None # 기본값을 None으로 설정
            # precision과 recall이 모두 수치일 경우에만 F1-score를 계산합니다.
            if precision is not None and recall is not None and (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)

            # --- 결과 출력 ---
            # 각 메트릭 값이 None이 아닐 경우에만 포맷팅하여 출력하고, 그렇지 않으면 'N/A'를 출력합니다.
            print(f"Accuracy: {accuracy:.4f}" if accuracy is not None else "Accuracy: N/A")
            print(f"Precision: {precision:.4f}" if precision is not None else "Precision: N/A")
            print(f"Recall: {recall:.4f}" if recall is not None else "Recall: N/A")
            print(f"F1-Score: {f1_score:.4f}" if f1_score is not None else "F1-Score: N/A")
            print(f"mAP50: {mAP50:.4f}" if mAP50 is not None else "mAP50: N/A")
            print(f"mAP (50-95): {mAP:.4f}" if mAP is not None else "mAP (50-95): N/A")
            print("------------------------")

        else:
            print("Could not retrieve validation metrics from training results.")

            

    except Exception as e:
        print(f"Error during YOLO training for method '{method}': {e}")


# --- 사용 예시 ---
if __name__ == "__main__":
    # 학습을 시작할 증강 방식 이름 지정 (예: 이전 증강 스크립트에서 사용한 blending mode와 동일하게)
    target_method = "train" # ⚠️ 여기에 실제로 사용한 증강 방식 폴더 이름을 입력하세요.

    train_yolo_for_method(target_method, epochs=30)

    print("\nAll training processes finished.")


--- Starting YOLOv8 training and validation for method: train ---
Loaded classes: {0: 'ac', 1: 'ph', 2: 'lc', 3: 'pc', 4: 'tc'}
Detected image dimensions: W=1920, H=648
Converting Labelme JSON to YOLO segmentation .txt for train...
Conversion complete.
'C:/Users/jenni/Desktop/final_data2\train\data.yaml' file created successfully.
Loaded YOLOv8n-seg model.
Ultralytics 8.3.152  Python-3.11.11 torch-2.7.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/jenni/Desktop/final_data2\train\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hs

train: Scanning C:\Users\jenni\Desktop\final_data2\train\labels.cache... 3000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3000/3000 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.1 ms, read: 288.3167.1 MB/s, size: 349.9 KB)



c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\jenni\Desktop\final_data2\train\labels.cache... 3000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3000/3000 [00:00<?, ?it/s]

Plotting labels to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL2\yolov8n_train4\labels.jpg... 



c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL2\yolov8n_train4
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/188 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
#조건2 클래스 근처 25%만 적용한 것+ 원본 파일 학습 (aug2)
import os
import cv2
import json
import glob
from ultralytics import YOLO


BASE_AUGMENTED_PATH = "C:/Users/jenni/Desktop/final_data1" # 증강된 데이터의 최상위 경로 (poisson_harmonized 폴더가 이 아래에 있음)
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_RESULTS_FINAL1" # YOLO 학습 결과가 저장될 경로


def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    os.makedirs(output_txt_dir, exist_ok=True)
    json_files = glob.glob(os.path.join(json_dir, "*.json"))

    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt_filename = os.path.basename(json_path).replace(".json", ".txt")
        txt_path = os.path.join(output_txt_dir, txt_filename)

        yolo_lines = []
        for shape in data['shapes']:
            label_name = shape['label']
            if label_name in label_to_id_map:
                class_id = label_to_id_map[label_name]
                points = shape['points']
                
                normalized_points = []
                for x, y in points:
                    # 좌표를 이미지 크기에 맞춰 정규화
                    normalized_points.append(f"{(x / img_w):.6f}")
                    normalized_points.append(f"{(y / img_h):.6f}")
                
                yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
            else:
                print(f"Warning: Class '{label_name}' not found in label_to_id_map. Skipping shape in {json_path}")

        if yolo_lines:
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(yolo_lines))
        # else:
        #    # 객체가 없는 이미지도 빈 .txt 파일로 저장하는 것이 YOLO 학습에 더 안전합니다.
        #    with open(txt_path, 'w') as f:
        #        pass # 빈 파일 생성


def create_data_yaml(dataset_base_path, yaml_output_path, class_names):
    
    images_path = os.path.join(dataset_base_path, "images")
    labels_path = os.path.join(dataset_base_path, "labels_yolo") # YOLO 형식의 라벨 폴더

    # 경로를 상대 경로가 아닌 절대 경로로 지정하여 혼란 방지
    train_img_path = images_path
    val_img_path = images_path # 검증 데이터가 따로 없으면 train과 동일하게 설정

    nc = len(class_names)
    names = class_names

    yaml_content = f"""
path: {dataset_base_path}
train: {os.path.relpath(train_img_path, dataset_base_path)} # path 기준으로 상대 경로
val: {os.path.relpath(val_img_path, dataset_base_path)}

nc: {nc}
names: {names}
"""
    # 실제 YOLO 라벨 경로를 data.yaml이 참조하도록 labels 폴더의 이름을 변경하거나 (labels -> labels_yolo),
    # data.yaml 내부에서 라벨 경로를 명시적으로 지정해야 합니다.
    # 일반적으로 YOLO는 'images' 폴더와 같은 레벨의 'labels' 폴더를 기대합니다.
    # 그래서 'labels_yolo'라는 폴더명을 가정했습니다.

    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' file created successfully.")


def train_yolo_for_method(method, epochs=50):
    print(f"\n--- Starting YOLOv8 training and validation for method: {method} ---")

    # 1. 각 증강 방식에 맞는 데이터셋 경로 설정
    dataset_path = os.path.join(BASE_AUGMENTED_PATH, method)
    images_path = os.path.join(dataset_path, "images")
    labels_json_path = os.path.join(dataset_path, "labels") # Labelme JSON 라벨이 있는 곳
    labels_yolo_path = os.path.join(dataset_path, "labels_yolo") # YOLO .txt 라벨을 저장할 곳

    # 2. 클래스 정보를 미리 정의
    label2id = {}
    id2label = {}
    try:
        with open("classes.txt", "r", encoding="utf-8") as f:
            class_names_from_file = [line.strip() for line in f if line.strip()]
        for i, name in enumerate(class_names_from_file):
            label2id[name] = i
            id2label[i] = name
        print(f"Loaded classes: {id2label}")
    except FileNotFoundError:
        print("Error: 'classes.txt' not found. Please ensure it's in the same directory or provide class info manually.")
        return
    except Exception as e:
        print(f"Error loading classes.txt: {e}")
        return

    # 이미지 크기를 추정하기 위해 첫 번째 이미지 파일을 읽습니다.
    first_image_path = None
    image_files = glob.glob(os.path.join(images_path, "*.jpg")) + glob.glob(os.path.join(images_path, "*.png"))
    if image_files:
        first_image_path = image_files[0]
        img_temp = cv2.imread(first_image_path)
        if img_temp is None:
            print(f"Error: Could not read image {first_image_path}. Check image file integrity.")
            return
        img_h, img_w = img_temp.shape[:2]
        print(f"Detected image dimensions: W={img_w}, H={img_h}")
    else:
        print(f"Error: No images found in {images_path}. Cannot determine image dimensions for YOLO conversion.")
        return

    # 3. Labelme JSON 라벨을 YOLO Segmentation .txt 형식으로 변환
    print(f"Converting Labelme JSON to YOLO segmentation .txt for {method}...")
    convert_labelme_to_yolo_segmentation(labels_json_path, labels_yolo_path, img_w, img_h, label2id)
    print("Conversion complete.")

    # 4. data.yaml 파일 생성
    yaml_path = os.path.join(dataset_path, "data.yaml")
    create_data_yaml(dataset_path, yaml_path, list(id2label.values()))

    # 5. 항상 동일한 사전 학습 모델로부터 학습 시작
    model = YOLO("yolov8n-seg.pt")
    print(f"Loaded YOLOv8n-seg model.")

    # 6. 모든 학습에 동일한 하이퍼파라미터 적용
    try:
        # 학습 수행
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_{method}",
            project=PROJECT_OUTPUT_PATH, # 결과 저장 최상위 경로
            save=True,
            resume=False
        )
        print(f"--- Training for method '{method}' completed. Results saved to {PROJECT_OUTPUT_PATH}/yolov8n_{method} ---")

        # 7. 학습 완료 후 검증 수행 (자동으로 수행되지만 명시적으로 호출 가능)
        
        if hasattr(results, 'metrics') and results.metrics:
            print("\n--- Validation Metrics ---")
            
            # Segmentation 모델의 경우 mAP는 'mAP50', 'mAP50-95' 형태로 접근합니다.
            # Precision, Recall, F1-score는 classification 또는 detection에서 더 흔하게 사용되지만,
            # segment 결과에서도 'metrics/precision(B)', 'metrics/recall(B)'와 같은 키로 접근 가능합니다.
            # F1-score는 Precision과 Recall로부터 계산해야 합니다.
            
            precision = results.metrics.get('metrics/precision(B)')
            recall = results.metrics.get('metrics/recall(B)')
            mAP50 = results.metrics.get('metrics/mAP50(B)')
            mAP = results.metrics.get('metrics/mAP50-95(B)')
            
            # [추가된 부분] 정확도(top-1 accuracy)를 가져옵니다.
            # 만약 사용하는 모델이 다른 키를 사용한다면 ('metrics/accuracy' 등) 이 부분을 수정해주세요.
            accuracy = results.metrics.get('metrics/accuracy_top1')

            # --- F1-score 계산 ---
            f1_score = None # 기본값을 None으로 설정
            # precision과 recall이 모두 수치일 경우에만 F1-score를 계산합니다.
            if precision is not None and recall is not None and (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)

            # --- 결과 출력 ---
            # 각 메트릭 값이 None이 아닐 경우에만 포맷팅하여 출력하고, 그렇지 않으면 'N/A'를 출력합니다.
            print(f"Accuracy: {accuracy:.4f}" if accuracy is not None else "Accuracy: N/A")
            print(f"Precision: {precision:.4f}" if precision is not None else "Precision: N/A")
            print(f"Recall: {recall:.4f}" if recall is not None else "Recall: N/A")
            print(f"F1-Score: {f1_score:.4f}" if f1_score is not None else "F1-Score: N/A")
            print(f"mAP50: {mAP50:.4f}" if mAP50 is not None else "mAP50: N/A")
            print(f"mAP (50-95): {mAP:.4f}" if mAP is not None else "mAP (50-95): N/A")
            print("------------------------")
        else:
            print("Could not retrieve validation metrics from training results.")

    except Exception as e:
        print(f"Error during YOLO training for method '{method}': {e}")


# --- 사용 예시 ---
if __name__ == "__main__":
    # 학습을 시작할 증강 방식 이름 지정 (예: 이전 증강 스크립트에서 사용한 blending mode와 동일하게)
    target_method = "train" # ⚠️ 여기에 실제로 사용한 증강 방식 폴더 이름을 입력하세요.

    train_yolo_for_method(target_method, epochs=50)

    print("\nAll training processes finished.")


--- Starting YOLOv8 training and validation for method: train ---
Loaded classes: {0: 'ac', 1: 'ph', 2: 'lc', 3: 'pc', 4: 'tc'}
Detected image dimensions: W=1920, H=648
Converting Labelme JSON to YOLO segmentation .txt for train...
Conversion complete.
'C:/Users/jenni/Desktop/final_data1\train\data.yaml' file created successfully.
Loaded YOLOv8n-seg model.
Ultralytics 8.3.152  Python-3.11.11 torch-2.7.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/jenni/Desktop/final_data1\train\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hs

train: Scanning C:\Users\jenni\Desktop\final_data1\train\labels... 2400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2400/2400 [00:04<00:00, 594.20it/s]


train: New cache created: C:\Users\jenni\Desktop\final_data1\train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 1364.9347.7 MB/s, size: 368.5 KB)


c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\jenni\Desktop\final_data1\train\labels.cache... 2400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2400/2400 [00:00<?, ?it/s]
c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL1\yolov8n_train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL1\yolov8n_train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   3%|▎         | 2/75 [00:03<02:22,  1.96s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 75/75 [01:57<00:00,  1.57s/it]


                   all       2400      17513      0.698      0.528      0.555      0.407      0.682      0.492      0.492      0.233
                    ac       1822       4104      0.871       0.91      0.942      0.786       0.85      0.874      0.878      0.467
                    ph       2152       4852      0.642      0.382      0.399      0.226      0.633      0.355      0.359      0.163
                    lc       1644       3226      0.777      0.827      0.845      0.682      0.745      0.767      0.742      0.316
                    pc       1385       2250      0.715      0.308      0.351      0.234       0.72      0.293      0.315      0.166
                    tc       1800       3081      0.483      0.215       0.24      0.108      0.461      0.172      0.167     0.0545
Speed: 0.7ms preprocess, 28.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL1\yolov8n_train
--- Training for method 'train' completed. Re

In [9]:
#final_data다 해보는 것

import os
import cv2
import json
import glob
from ultralytics import YOLO

# --- 경로 설정 ---
# [기존] 학습에 사용할 증강 데이터셋 경로
BASE_AUGMENTED_PATH = "C:/Users/jenni/Desktop/final_data6"
# [변경] 검증에 사용할 원본 데이터셋 경로
VALIDATION_DATA_PATH = "C:/Users/jenni/Desktop/Final data and augmented datasets/val" # ⚠️ 검증 데이터 폴더 경로를 정확히 지정하세요.
# [기존] YOLO 학습 결과 저장 경로
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_RESULTS_FINAL6"


def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    """LabelMe JSON 파일을 YOLO Segmentation TXT 파일로 변환합니다."""
    os.makedirs(output_txt_dir, exist_ok=True)
    
    # JSON 파일 검색 개선 - 대소문자 구분 없이 검색
    json_patterns = [
        os.path.join(json_dir, "*.json"),
        os.path.join(json_dir, "*.JSON")
    ]
    
    json_files = []
    for pattern in json_patterns:
        json_files.extend(glob.glob(pattern))
    
    # 중복 제거
    json_files = list(set(json_files))
    
    print(f"JSON 폴더 경로: {json_dir}")
    print(f"찾은 JSON 파일 개수: {len(json_files)}")
    
    if not json_files:
        print(f"Warning: No JSON files found in {json_dir}")
        # 디렉토리 내 모든 파일 목록 출력
        all_files = os.listdir(json_dir) if os.path.exists(json_dir) else []
        print(f"디렉토리 내 모든 파일: {all_files}")
        return

    converted_count = 0
    error_count = 0
    
    for json_path in json_files:
        try:
            print(f"처리 중: {json_path}")
            
            # JSON 파일 읽기 시도
            with open(json_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"JSON 파싱 오류 in {json_path}: {e}")
                    # UTF-8로 실패하면 다른 인코딩 시도
                    try:
                        with open(json_path, 'r', encoding='cp949') as f2:
                            data = json.load(f2)
                        print(f"CP949 인코딩으로 성공적으로 읽음: {json_path}")
                    except:
                        with open(json_path, 'r', encoding='latin-1') as f3:
                            data = json.load(f3)
                        print(f"Latin-1 인코딩으로 성공적으로 읽음: {json_path}")

            # 파일명 생성
            txt_filename = os.path.basename(json_path).replace(".json", ".txt").replace(".JSON", ".txt")
            txt_path = os.path.join(output_txt_dir, txt_filename)

            yolo_lines = []
            
            # shapes 키가 존재하는지 확인
            if 'shapes' not in data:
                print(f"Warning: 'shapes' key not found in {json_path}")
                print(f"JSON 파일의 키들: {list(data.keys())}")
                continue
                
            shapes = data.get('shapes', [])
            print(f"  - 발견된 shape 개수: {len(shapes)}")
            
            for i, shape in enumerate(shapes):
                try:
                    label_name = shape.get('label', '').strip()
                    if not label_name:
                        print(f"    Warning: Empty label in shape {i} of {json_path}")
                        continue
                        
                    if label_name in label_to_id_map:
                        class_id = label_to_id_map[label_name]
                        points = shape.get('points', [])
                        
                        if not points:
                            print(f"    Warning: No points in shape {i} of {json_path}")
                            continue
                        
                        # 포인트 정규화
                        normalized_points = []
                        for point in points:
                            if len(point) >= 2:
                                x, y = point[0], point[1]
                                # 좌표값이 이미지 크기를 벗어나지 않도록 클리핑
                                x = max(0, min(x, img_w))
                                y = max(0, min(y, img_h))
                                normalized_points.append(f"{(x / img_w):.6f}")
                                normalized_points.append(f"{(y / img_h):.6f}")
                            else:
                                print(f"    Warning: Invalid point format in shape {i} of {json_path}")
                        
                        if normalized_points:
                            yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
                            print(f"    - 변환 완료: {label_name} (class_id: {class_id})")
                        else:
                            print(f"    Warning: No valid points for shape {i} in {json_path}")
                            
                    else:
                        print(f"    Warning: 클래스 '{label_name}'가 label_to_id_map에 없습니다. (파일: {json_path})")
                        print(f"    사용 가능한 클래스: {list(label_to_id_map.keys())}")
                        
                except Exception as e:
                    print(f"    Error processing shape {i} in {json_path}: {e}")
                    continue

            # TXT 파일 저장
            if yolo_lines:
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write('\n'.join(yolo_lines))
                converted_count += 1
                print(f"  저장 완료: {txt_path}")
            else:
                print(f"  Warning: No valid annotations found in {json_path}")
                # 빈 파일이라도 생성
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write('')
                    
        except Exception as e:
            print(f"Error processing {json_path}: {e}")
            error_count += 1
            continue
    
    print(f"변환 완료: {converted_count}개 파일, 오류: {error_count}개 파일")


def create_data_yaml(yaml_output_path, train_dataset_path, val_dataset_path, class_names):
    """학습 및 검증 경로를 지정하여 data.yaml 파일을 생성합니다."""
    
    nc = len(class_names)
    
    # YOLO는 데이터셋의 루트 경로를 기대합니다 (images와 labels 폴더의 상위 폴더)
    train_path_yaml = train_dataset_path.replace('\\', '/')
    val_path_yaml = val_dataset_path.replace('\\', '/')

    yaml_content = f"""# Train/val data paths
train: {train_path_yaml}
val: {val_path_yaml}

# Number of classes and names
nc: {nc}
names: {class_names}
"""

    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' 파일이 성공적으로 생성되었습니다.")
    print(f"Train path: {train_path_yaml}")
    print(f"Val path: {val_path_yaml}")


def load_classes():
    """classes.txt 파일에서 클래스 정보를 로드합니다."""
    label2id = {}
    id2label = {}
    
    classes_file_paths = [
        "classes.txt",
        os.path.join(BASE_AUGMENTED_PATH, "classes.txt"),
        os.path.join(VALIDATION_DATA_PATH, "classes.txt")
    ]
    
    classes_loaded = False
    for classes_path in classes_file_paths:
        if os.path.exists(classes_path):
            try:
                print(f"클래스 파일 로드 시도: {classes_path}")
                with open(classes_path, "r", encoding="utf-8") as f:
                    class_names_from_file = [line.strip() for line in f if line.strip()]
                
                if class_names_from_file:
                    for i, name in enumerate(class_names_from_file):
                        label2id[name] = i
                        id2label[i] = name
                    print(f"클래스 로드 완료: {id2label}")
                    classes_loaded = True
                    break
                    
            except Exception as e:
                print(f"클래스 파일 읽기 오류 ({classes_path}): {e}")
                continue
    
    if not classes_loaded:
        print("Error: classes.txt 파일을 찾을 수 없습니다.")
        print("다음 위치 중 하나에 파일이 있어야 합니다:")
        for path in classes_file_paths:
            print(f"  - {path}")
        return None, None
        
    return label2id, id2label


def verify_dataset_structure(images_path, labels_path):
    """데이터셋 구조가 올바른지 확인합니다."""
    print(f"\n=== 데이터셋 구조 확인 ===")
    print(f"이미지 경로: {images_path}")
    print(f"라벨 경로: {labels_path}")
    
    # 경로 존재 확인
    if not os.path.exists(images_path):
        print(f"오류: 이미지 폴더가 존재하지 않습니다: {images_path}")
        return False
    
    if not os.path.exists(labels_path):
        print(f"오류: 라벨 폴더가 존재하지 않습니다: {labels_path}")
        return False
    
    # 파일 개수 확인
    image_files = glob.glob(os.path.join(images_path, "*.jpg")) + glob.glob(os.path.join(images_path, "*.png"))
    json_files = glob.glob(os.path.join(labels_path, "*.json")) + glob.glob(os.path.join(labels_path, "*.JSON"))
    
    print(f"이미지 파일 개수: {len(image_files)}")
    print(f"JSON 라벨 파일 개수: {len(json_files)}")
    
    if len(image_files) == 0:
        print("오류: 이미지 파일이 없습니다.")
        return False
    
    if len(json_files) == 0:
        print("오류: JSON 라벨 파일이 없습니다.")
        return False
    
    print("데이터셋 구조 확인 완료")
    return True


def train_yolo_for_method(method, epochs=50):
    print(f"\n--- Starting YOLOv8 training and validation for method: {method} ---")

    # 1. 학습(Augmented) 및 검증(Raw) 데이터셋 경로 설정
    train_dataset_path = os.path.join(BASE_AUGMENTED_PATH, method)
    train_images_path = os.path.join(train_dataset_path, "images")
    train_labels_json_path = os.path.join(train_dataset_path, "labels")
    train_labels_yolo_path = os.path.join(train_dataset_path, "labels_yolo")

    # [변경] 검증 데이터셋 경로 설정
    val_images_path = os.path.join(VALIDATION_DATA_PATH, "images")
    val_labels_json_path = os.path.join(VALIDATION_DATA_PATH, "labels")
    val_labels_yolo_path = os.path.join(VALIDATION_DATA_PATH, "labels_yolo")

    # 2. 데이터셋 구조 확인
    print("=== 학습 데이터셋 확인 ===")
    if not verify_dataset_structure(train_images_path, train_labels_json_path):
        return
    
    print("=== 검증 데이터셋 확인 ===")
    if not verify_dataset_structure(val_images_path, val_labels_json_path):
        return

    # 3. 클래스 정보 로드
    label2id, id2label = load_classes()
    if label2id is None:
        return

    # 4. 라벨 변환 (학습 데이터셋)
    train_image_files = glob.glob(os.path.join(train_images_path, "*.jpg")) + glob.glob(os.path.join(train_images_path, "*.png"))
    if not train_image_files:
        print(f"Error: No training images found in {train_images_path}.")
        return
    
    train_img_temp = cv2.imread(train_image_files[0])
    if train_img_temp is None:
        print(f"Error: Cannot read image {train_image_files[0]}")
        return
    
    train_h, train_w = train_img_temp.shape[:2]
    print(f"학습 이미지 크기: {train_w}x{train_h}")
    print(f"Converting training labels for {method}...")
    convert_labelme_to_yolo_segmentation(train_labels_json_path, train_labels_yolo_path, train_w, train_h, label2id)
    print("Training label conversion complete.")

    # 5. 라벨 변환 (검증 데이터셋)
    val_image_files = glob.glob(os.path.join(val_images_path, "*.jpg")) + glob.glob(os.path.join(val_images_path, "*.png"))
    if not val_image_files:
        print(f"Error: No validation images found in {val_images_path}.")
        return
    
    val_img_temp = cv2.imread(val_image_files[0])
    if val_img_temp is None:
        print(f"Error: Cannot read image {val_image_files[0]}")
        return
    
    val_h, val_w = val_img_temp.shape[:2]
    print(f"검증 이미지 크기: {val_w}x{val_h}")
    print(f"Converting validation labels...")
    convert_labelme_to_yolo_segmentation(val_labels_json_path, val_labels_yolo_path, val_w, val_h, label2id)
    print("Validation label conversion complete.")
    
    # ⭐ YOLO 형식으로 변환된 라벨을 기존 labels 폴더로 복사
    print("YOLO 라벨을 기존 labels 폴더로 복사 중...")
    
    # 학습 데이터 라벨 복사
    train_yolo_txt_files = glob.glob(os.path.join(train_labels_yolo_path, "*.txt"))
    for txt_file in train_yolo_txt_files:
        filename = os.path.basename(txt_file)
        dest_path = os.path.join(train_labels_json_path, filename)
        try:
            with open(txt_file, 'r', encoding='utf-8') as src:
                content = src.read()
            with open(dest_path, 'w', encoding='utf-8') as dst:
                dst.write(content)
        except Exception as e:
            print(f"라벨 복사 오류 ({txt_file}): {e}")
    
    # 검증 데이터 라벨 복사
    val_yolo_txt_files = glob.glob(os.path.join(val_labels_yolo_path, "*.txt"))
    for txt_file in val_yolo_txt_files:
        filename = os.path.basename(txt_file)
        dest_path = os.path.join(val_labels_json_path, filename)
        try:
            with open(txt_file, 'r', encoding='utf-8') as src:
                content = src.read()
            with open(dest_path, 'w', encoding='utf-8') as dst:
                dst.write(content)
        except Exception as e:
            print(f"라벨 복사 오류 ({txt_file}): {e}")
    
    print(f"라벨 복사 완료 - 학습: {len(train_yolo_txt_files)}개, 검증: {len(val_yolo_txt_files)}개")
    
    # 변환 결과 확인
    print("\n=== 변환 결과 확인 ===")
    sample_txt_files = glob.glob(os.path.join(train_labels_json_path, "*.txt"))[:3]
    for txt_file in sample_txt_files:
        print(f"샘플 파일: {os.path.basename(txt_file)}")
        try:
            with open(txt_file, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                if content:
                    lines = content.split('\n')
                    print(f"  라벨 개수: {len(lines)}")
                    print(f"  첫 번째 라벨: {lines[0]}")
                else:
                    print("  빈 파일")
        except Exception as e:
            print(f"  파일 읽기 오류: {e}")
    
    # 6. data.yaml 파일 생성 (데이터셋 루트 경로로 수정)
    yaml_path = os.path.join(PROJECT_OUTPUT_PATH, f"data_{method}.yaml")
    os.makedirs(PROJECT_OUTPUT_PATH, exist_ok=True)
    create_data_yaml(yaml_path, train_dataset_path, os.path.dirname(val_images_path), list(id2label.values()))

    # 7. YOLO 모델 로드 및 학습
    try:
        model = YOLO("yolov8n-seg.pt")
        print(f"Loaded YOLOv8n-seg model.")

        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_{method}",
            project=PROJECT_OUTPUT_PATH,
            save=True,
            resume=False,
            verbose=True
        )
        print(f"--- Training for method '{method}' completed. Results saved to {PROJECT_OUTPUT_PATH}/yolov8n_{method} ---")

        # 8. 검증 결과 출력
        if hasattr(results, 'metrics') and results.metrics:
            print("\n--- Final Validation Metrics ---")
            try:
                precision = results.metrics.get('metrics/precision(B)')
                recall = results.metrics.get('metrics/recall(B)')
                mAP50 = results.metrics.get('metrics/mAP50(B)')
                mAP = results.metrics.get('metrics/mAP50-95(B)')
                accuracy = results.metrics.get('metrics/accuracy_top1')
                
                f1_score = None
                if precision is not None and recall is not None and (precision + recall) > 0:
                    f1_score = 2 * (precision * recall) / (precision + recall)

                print(f"Accuracy: {accuracy:.4f}" if accuracy is not None else "Accuracy: N/A")
                print(f"Precision: {precision:.4f}" if precision is not None else "Precision: N/A")
                print(f"Recall: {recall:.4f}" if recall is not None else "Recall: N/A")
                print(f"F1-Score: {f1_score:.4f}" if f1_score is not None else "F1-Score: N/A")
                print(f"mAP50: {mAP50:.4f}" if mAP50 is not None else "mAP50: N/A")
                print(f"mAP (50-95): {mAP:.4f}" if mAP is not None else "mAP (50-95): N/A")
                print("------------------------")
            except Exception as e:
                print(f"메트릭 출력 중 오류: {e}")
        else:
            print("Could not retrieve validation metrics from training results.")

    except Exception as e:
        print(f"Error during YOLO training for method '{method}': {e}")
        import traceback
        traceback.print_exc()


# --- 사용 예시 ---
if __name__ == "__main__":
    # 학습을 시작할 증강 데이터가 있는 폴더 이름
    target_method = "train" 
    train_yolo_for_method(target_method, epochs=50)

    print("\nAll training processes finished.")


--- Starting YOLOv8 training and validation for method: train ---
=== 학습 데이터셋 확인 ===

=== 데이터셋 구조 확인 ===
이미지 경로: C:/Users/jenni/Desktop/final_data6\train\images
라벨 경로: C:/Users/jenni/Desktop/final_data6\train\labels
이미지 파일 개수: 3000
JSON 라벨 파일 개수: 6000
데이터셋 구조 확인 완료
=== 검증 데이터셋 확인 ===

=== 데이터셋 구조 확인 ===
이미지 경로: C:/Users/jenni/Desktop/Final data and augmented datasets/val\images
라벨 경로: C:/Users/jenni/Desktop/Final data and augmented datasets/val\labels
이미지 파일 개수: 255
JSON 라벨 파일 개수: 510
데이터셋 구조 확인 완료
클래스 파일 로드 시도: classes.txt
클래스 로드 완료: {0: 'ac', 1: 'ph', 2: 'lc', 3: 'pc', 4: 'tc'}
학습 이미지 크기: 1920x648
Converting training labels for train...
JSON 폴더 경로: C:/Users/jenni/Desktop/final_data6\train\labels
찾은 JSON 파일 개수: 3000
처리 중: C:/Users/jenni/Desktop/final_data6\train\labels\aug_9e239632a37e.json
  - 발견된 shape 개수: 11
    - 변환 완료: ac (class_id: 0)
    - 변환 완료: lc (class_id: 2)
    - 변환 완료: lc (class_id: 2)
    - 변환 완료: ac (class_id: 0)
    - 변환 완료: ac (class_id: 0)
    - 변환 완료: lc (class_id

train: Scanning C:\Users\jenni\Desktop\final_data6\train\labels.cache... 3000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3000/3000 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 728.8164.0 MB/s, size: 153.1 KB)



c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\jenni\Desktop\Final data and augmented datasets\val\labels.cache... 255 images, 6 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]

Plotting labels to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL6\yolov8n_train\labels.jpg... 


...

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:03<00:09,  1.64s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:11<00:00,  1.45s/it]


                   all        255        863       0.61      0.528      0.576      0.365      0.588      0.492      0.523      0.245
                    ac        185        391      0.769      0.739      0.813      0.523       0.76      0.691       0.73      0.365
                    ph         19         27      0.446       0.37      0.419      0.213      0.381      0.296      0.348     0.0819
                    lc        162        316      0.681       0.67      0.698      0.501      0.696      0.655      0.673      0.268
                    pc         54         89      0.717      0.713      0.766       0.52      0.747      0.708      0.742      0.459
                    tc         35         40      0.436       0.15      0.184     0.0665      0.359      0.112      0.121     0.0491
Speed: 0.7ms preprocess, 29.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL6\yolov8n_train
--- Training for method 'train' completed. Re

In [12]:
import os
import cv2
import json
import glob
from ultralytics import YOLO

# --- 경로 설정 ---
# [기존] 학습에 사용할 증강 데이터셋 경로
BASE_AUGMENTED_PATH = "C:/Users/jenni/Desktop/final_data7"
# [변경] 검증에 사용할 원본 데이터셋 경로
VALIDATION_DATA_PATH = "C:/Users/jenni/Desktop/Final data and augmented datasets/val" # ⚠️ 검증 데이터 폴더 경로를 정확히 지정하세요.
# [기존] YOLO 학습 결과 저장 경로
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_RESULTS_FINAL7"


def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    """LabelMe JSON 파일을 YOLO Segmentation TXT 파일로 변환합니다."""
    os.makedirs(output_txt_dir, exist_ok=True)
    
    # JSON 파일 검색 개선 - 대소문자 구분 없이 검색
    json_patterns = [
        os.path.join(json_dir, "*.json"),
        os.path.join(json_dir, "*.JSON")
    ]
    
    json_files = []
    for pattern in json_patterns:
        json_files.extend(glob.glob(pattern))
    
    # 중복 제거
    json_files = list(set(json_files))
    
    print(f"JSON 폴더 경로: {json_dir}")
    print(f"찾은 JSON 파일 개수: {len(json_files)}")
    
    if not json_files:
        print(f"Warning: No JSON files found in {json_dir}")
        # 디렉토리 내 모든 파일 목록 출력
        all_files = os.listdir(json_dir) if os.path.exists(json_dir) else []
        print(f"디렉토리 내 모든 파일: {all_files}")
        return

    converted_count = 0
    error_count = 0
    
    for json_path in json_files:
        try:
            print(f"처리 중: {json_path}")
            
            # JSON 파일 읽기 시도
            with open(json_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"JSON 파싱 오류 in {json_path}: {e}")
                    # UTF-8로 실패하면 다른 인코딩 시도
                    try:
                        with open(json_path, 'r', encoding='cp949') as f2:
                            data = json.load(f2)
                        print(f"CP949 인코딩으로 성공적으로 읽음: {json_path}")
                    except:
                        with open(json_path, 'r', encoding='latin-1') as f3:
                            data = json.load(f3)
                        print(f"Latin-1 인코딩으로 성공적으로 읽음: {json_path}")

            # 파일명 생성
            txt_filename = os.path.basename(json_path).replace(".json", ".txt").replace(".JSON", ".txt")
            txt_path = os.path.join(output_txt_dir, txt_filename)

            yolo_lines = []
            
            # shapes 키가 존재하는지 확인
            if 'shapes' not in data:
                print(f"Warning: 'shapes' key not found in {json_path}")
                print(f"JSON 파일의 키들: {list(data.keys())}")
                continue
                
            shapes = data.get('shapes', [])
            print(f"  - 발견된 shape 개수: {len(shapes)}")
            
            for i, shape in enumerate(shapes):
                try:
                    label_name = shape.get('label', '').strip()
                    if not label_name:
                        print(f"    Warning: Empty label in shape {i} of {json_path}")
                        continue
                        
                    if label_name in label_to_id_map:
                        class_id = label_to_id_map[label_name]
                        points = shape.get('points', [])
                        
                        if not points:
                            print(f"    Warning: No points in shape {i} of {json_path}")
                            continue
                        
                        # 포인트 정규화
                        normalized_points = []
                        for point in points:
                            if len(point) >= 2:
                                x, y = point[0], point[1]
                                # 좌표값이 이미지 크기를 벗어나지 않도록 클리핑
                                x = max(0, min(x, img_w))
                                y = max(0, min(y, img_h))
                                normalized_points.append(f"{(x / img_w):.6f}")
                                normalized_points.append(f"{(y / img_h):.6f}")
                            else:
                                print(f"    Warning: Invalid point format in shape {i} of {json_path}")
                        
                        if normalized_points:
                            yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
                            print(f"    - 변환 완료: {label_name} (class_id: {class_id})")
                        else:
                            print(f"    Warning: No valid points for shape {i} in {json_path}")
                            
                    else:
                        print(f"    Warning: 클래스 '{label_name}'가 label_to_id_map에 없습니다. (파일: {json_path})")
                        print(f"    사용 가능한 클래스: {list(label_to_id_map.keys())}")
                        
                except Exception as e:
                    print(f"    Error processing shape {i} in {json_path}: {e}")
                    continue

            # TXT 파일 저장
            if yolo_lines:
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write('\n'.join(yolo_lines))
                converted_count += 1
                print(f"  저장 완료: {txt_path}")
            else:
                print(f"  Warning: No valid annotations found in {json_path}")
                # 빈 파일이라도 생성
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write('')
                    
        except Exception as e:
            print(f"Error processing {json_path}: {e}")
            error_count += 1
            continue
    
    print(f"변환 완료: {converted_count}개 파일, 오류: {error_count}개 파일")


def create_data_yaml(yaml_output_path, train_dataset_path, val_dataset_path, class_names):
    """학습 및 검증 경로를 지정하여 data.yaml 파일을 생성합니다."""
    
    nc = len(class_names)
    
    # YOLO는 데이터셋의 루트 경로를 기대합니다 (images와 labels 폴더의 상위 폴더)
    train_path_yaml = train_dataset_path.replace('\\', '/')
    val_path_yaml = val_dataset_path.replace('\\', '/')

    yaml_content = f"""# Train/val data paths
train: {train_path_yaml}
val: {val_path_yaml}

# Number of classes and names
nc: {nc}
names: {class_names}
"""

    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' 파일이 성공적으로 생성되었습니다.")
    print(f"Train path: {train_path_yaml}")
    print(f"Val path: {val_path_yaml}")


def load_classes():
    """classes.txt 파일에서 클래스 정보를 로드합니다."""
    label2id = {}
    id2label = {}
    
    classes_file_paths = [
        "classes.txt",
        os.path.join(BASE_AUGMENTED_PATH, "classes.txt"),
        os.path.join(VALIDATION_DATA_PATH, "classes.txt")
    ]
    
    classes_loaded = False
    for classes_path in classes_file_paths:
        if os.path.exists(classes_path):
            try:
                print(f"클래스 파일 로드 시도: {classes_path}")
                with open(classes_path, "r", encoding="utf-8") as f:
                    class_names_from_file = [line.strip() for line in f if line.strip()]
                
                if class_names_from_file:
                    for i, name in enumerate(class_names_from_file):
                        label2id[name] = i
                        id2label[i] = name
                    print(f"클래스 로드 완료: {id2label}")
                    classes_loaded = True
                    break
                    
            except Exception as e:
                print(f"클래스 파일 읽기 오류 ({classes_path}): {e}")
                continue
    
    if not classes_loaded:
        print("Error: classes.txt 파일을 찾을 수 없습니다.")
        print("다음 위치 중 하나에 파일이 있어야 합니다:")
        for path in classes_file_paths:
            print(f"  - {path}")
        return None, None
        
    return label2id, id2label


def verify_dataset_structure(images_path, labels_path):
    """데이터셋 구조가 올바른지 확인합니다."""
    print(f"\n=== 데이터셋 구조 확인 ===")
    print(f"이미지 경로: {images_path}")
    print(f"라벨 경로: {labels_path}")
    
    # 경로 존재 확인
    if not os.path.exists(images_path):
        print(f"오류: 이미지 폴더가 존재하지 않습니다: {images_path}")
        return False
    
    if not os.path.exists(labels_path):
        print(f"오류: 라벨 폴더가 존재하지 않습니다: {labels_path}")
        return False
    
    # 파일 개수 확인
    image_files = glob.glob(os.path.join(images_path, "*.jpg")) + glob.glob(os.path.join(images_path, "*.png"))
    json_files = glob.glob(os.path.join(labels_path, "*.json")) + glob.glob(os.path.join(labels_path, "*.JSON"))
    
    print(f"이미지 파일 개수: {len(image_files)}")
    print(f"JSON 라벨 파일 개수: {len(json_files)}")
    
    if len(image_files) == 0:
        print("오류: 이미지 파일이 없습니다.")
        return False
    
    if len(json_files) == 0:
        print("오류: JSON 라벨 파일이 없습니다.")
        return False
    
    print("데이터셋 구조 확인 완료")
    return True


def train_yolo_for_method(method, epochs=50):
    print(f"\n--- Starting YOLOv8 training and validation for method: {method} ---")

    # 1. 학습(Augmented) 및 검증(Raw) 데이터셋 경로 설정
    train_dataset_path = os.path.join(BASE_AUGMENTED_PATH, method)
    train_images_path = os.path.join(train_dataset_path, "images")
    train_labels_json_path = os.path.join(train_dataset_path, "labels")
    train_labels_yolo_path = os.path.join(train_dataset_path, "labels_yolo")

    # [변경] 검증 데이터셋 경로 설정
    val_images_path = os.path.join(VALIDATION_DATA_PATH, "images")
    val_labels_json_path = os.path.join(VALIDATION_DATA_PATH, "labels")
    val_labels_yolo_path = os.path.join(VALIDATION_DATA_PATH, "labels_yolo")

    # 2. 데이터셋 구조 확인
    print("=== 학습 데이터셋 확인 ===")
    if not verify_dataset_structure(train_images_path, train_labels_json_path):
        return
    
    print("=== 검증 데이터셋 확인 ===")
    if not verify_dataset_structure(val_images_path, val_labels_json_path):
        return

    # 3. 클래스 정보 로드
    label2id, id2label = load_classes()
    if label2id is None:
        return

    # 4. 라벨 변환 (학습 데이터셋)
    train_image_files = glob.glob(os.path.join(train_images_path, "*.jpg")) + glob.glob(os.path.join(train_images_path, "*.png"))
    if not train_image_files:
        print(f"Error: No training images found in {train_images_path}.")
        return
    
    train_img_temp = cv2.imread(train_image_files[0])
    if train_img_temp is None:
        print(f"Error: Cannot read image {train_image_files[0]}")
        return
    
    train_h, train_w = train_img_temp.shape[:2]
    print(f"학습 이미지 크기: {train_w}x{train_h}")
    print(f"Converting training labels for {method}...")
    convert_labelme_to_yolo_segmentation(train_labels_json_path, train_labels_yolo_path, train_w, train_h, label2id)
    print("Training label conversion complete.")

    # 5. 라벨 변환 (검증 데이터셋)
    val_image_files = glob.glob(os.path.join(val_images_path, "*.jpg")) + glob.glob(os.path.join(val_images_path, "*.png"))
    if not val_image_files:
        print(f"Error: No validation images found in {val_images_path}.")
        return
    
    val_img_temp = cv2.imread(val_image_files[0])
    if val_img_temp is None:
        print(f"Error: Cannot read image {val_image_files[0]}")
        return
    
    val_h, val_w = val_img_temp.shape[:2]
    print(f"검증 이미지 크기: {val_w}x{val_h}")
    print(f"Converting validation labels...")
    convert_labelme_to_yolo_segmentation(val_labels_json_path, val_labels_yolo_path, val_w, val_h, label2id)
    print("Validation label conversion complete.")
    
    # ⭐ YOLO 형식으로 변환된 라벨을 기존 labels 폴더로 복사
    print("YOLO 라벨을 기존 labels 폴더로 복사 중...")
    
    # 학습 데이터 라벨 복사
    train_yolo_txt_files = glob.glob(os.path.join(train_labels_yolo_path, "*.txt"))
    for txt_file in train_yolo_txt_files:
        filename = os.path.basename(txt_file)
        dest_path = os.path.join(train_labels_json_path, filename)
        try:
            with open(txt_file, 'r', encoding='utf-8') as src:
                content = src.read()
            with open(dest_path, 'w', encoding='utf-8') as dst:
                dst.write(content)
        except Exception as e:
            print(f"라벨 복사 오류 ({txt_file}): {e}")
    
    # 검증 데이터 라벨 복사
    val_yolo_txt_files = glob.glob(os.path.join(val_labels_yolo_path, "*.txt"))
    for txt_file in val_yolo_txt_files:
        filename = os.path.basename(txt_file)
        dest_path = os.path.join(val_labels_json_path, filename)
        try:
            with open(txt_file, 'r', encoding='utf-8') as src:
                content = src.read()
            with open(dest_path, 'w', encoding='utf-8') as dst:
                dst.write(content)
        except Exception as e:
            print(f"라벨 복사 오류 ({txt_file}): {e}")
    
    print(f"라벨 복사 완료 - 학습: {len(train_yolo_txt_files)}개, 검증: {len(val_yolo_txt_files)}개")
    
    # 변환 결과 확인
    print("\n=== 변환 결과 확인 ===")
    sample_txt_files = glob.glob(os.path.join(train_labels_json_path, "*.txt"))[:3]
    for txt_file in sample_txt_files:
        print(f"샘플 파일: {os.path.basename(txt_file)}")
        try:
            with open(txt_file, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                if content:
                    lines = content.split('\n')
                    print(f"  라벨 개수: {len(lines)}")
                    print(f"  첫 번째 라벨: {lines[0]}")
                else:
                    print("  빈 파일")
        except Exception as e:
            print(f"  파일 읽기 오류: {e}")
    
    # 6. data.yaml 파일 생성 (데이터셋 루트 경로로 수정)
    yaml_path = os.path.join(PROJECT_OUTPUT_PATH, f"data_{method}.yaml")
    os.makedirs(PROJECT_OUTPUT_PATH, exist_ok=True)
    create_data_yaml(yaml_path, train_dataset_path, os.path.dirname(val_images_path), list(id2label.values()))

    # 7. YOLO 모델 로드 및 학습
    try:
        model = YOLO("yolov8n-seg.pt")
        print(f"Loaded YOLOv8n-seg model.")

        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_{method}",
            project=PROJECT_OUTPUT_PATH,
            save=True,
            resume=False,
            verbose=True
        )
        print(f"--- Training for method '{method}' completed. Results saved to {PROJECT_OUTPUT_PATH}/yolov8n_{method} ---")

        # 8. 검증 결과 출력
        if hasattr(results, 'metrics') and results.metrics:
            print("\n--- Final Validation Metrics ---")
            try:
                precision = results.metrics.get('metrics/precision(B)')
                recall = results.metrics.get('metrics/recall(B)')
                mAP50 = results.metrics.get('metrics/mAP50(B)')
                mAP = results.metrics.get('metrics/mAP50-95(B)')
                accuracy = results.metrics.get('metrics/accuracy_top1')
                
                f1_score = None
                if precision is not None and recall is not None and (precision + recall) > 0:
                    f1_score = 2 * (precision * recall) / (precision + recall)

                print(f"Accuracy: {accuracy:.4f}" if accuracy is not None else "Accuracy: N/A")
                print(f"Precision: {precision:.4f}" if precision is not None else "Precision: N/A")
                print(f"Recall: {recall:.4f}" if recall is not None else "Recall: N/A")
                print(f"F1-Score: {f1_score:.4f}" if f1_score is not None else "F1-Score: N/A")
                print(f"mAP50: {mAP50:.4f}" if mAP50 is not None else "mAP50: N/A")
                print(f"mAP (50-95): {mAP:.4f}" if mAP is not None else "mAP (50-95): N/A")
                print("------------------------")
            except Exception as e:
                print(f"메트릭 출력 중 오류: {e}")
        else:
            print("Could not retrieve validation metrics from training results.")

    except Exception as e:
        print(f"Error during YOLO training for method '{method}': {e}")
        import traceback
        traceback.print_exc()


# --- 사용 예시 ---
if __name__ == "__main__":
    # 학습을 시작할 증강 데이터가 있는 폴더 이름
    target_method = "train" 
    train_yolo_for_method(target_method, epochs=50)

    print("\nAll training processes finished.")


--- Starting YOLOv8 training and validation for method: train ---
=== 학습 데이터셋 확인 ===

=== 데이터셋 구조 확인 ===
이미지 경로: C:/Users/jenni/Desktop/final_data7\train\images
라벨 경로: C:/Users/jenni/Desktop/final_data7\train\labels
이미지 파일 개수: 3000
JSON 라벨 파일 개수: 6000
데이터셋 구조 확인 완료
=== 검증 데이터셋 확인 ===

=== 데이터셋 구조 확인 ===
이미지 경로: C:/Users/jenni/Desktop/Final data and augmented datasets/val\images
라벨 경로: C:/Users/jenni/Desktop/Final data and augmented datasets/val\labels
이미지 파일 개수: 255
JSON 라벨 파일 개수: 510
데이터셋 구조 확인 완료
클래스 파일 로드 시도: classes.txt
클래스 로드 완료: {0: 'ac', 1: 'ph', 2: 'lc', 3: 'pc', 4: 'tc'}
학습 이미지 크기: 1920x648
Converting training labels for train...
JSON 폴더 경로: C:/Users/jenni/Desktop/final_data7\train\labels
찾은 JSON 파일 개수: 3000
처리 중: C:/Users/jenni/Desktop/final_data7\train\labels\aug_6f857a7337fb.json
  - 발견된 shape 개수: 9
    - 변환 완료: lc (class_id: 2)
    - 변환 완료: lc (class_id: 2)
    - 변환 완료: tc (class_id: 4)
    - 변환 완료: tc (class_id: 4)
    - 변환 완료: ac (class_id: 0)
    - 변환 완료: tc (class_id:

train: Scanning C:\Users\jenni\Desktop\final_data7\train\labels... 3000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3000/3000 [00:04<00:00, 727.45it/s]


train: New cache created: C:\Users\jenni\Desktop\final_data7\train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 17.54.6 MB/s, size: 153.1 KB)


c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\jenni\Desktop\Final data and augmented datasets\val\labels.cache... 255 images, 6 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]

Plotting labels to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL7\yolov8n_train\labels.jpg... 


...

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:03<00:10,  1.67s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:12<00:00,  1.54s/it]


                   all        255        863      0.581       0.55      0.566      0.356      0.553      0.523      0.523      0.244
                    ac        185        391       0.79      0.714      0.799      0.519      0.734      0.662      0.685      0.342
                    ph         19         27      0.378      0.407      0.363      0.182      0.309      0.333      0.317      0.104
                    lc        162        316       0.71       0.69      0.699      0.489      0.697      0.678      0.673      0.268
                    pc         54         89      0.685      0.764      0.766      0.501      0.685      0.764      0.754      0.437
                    tc         35         40      0.341      0.175      0.204     0.0871      0.341      0.175      0.185     0.0677
Speed: 0.8ms preprocess, 32.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to C:\Users\jenni\Desktop\YOLO_RESULTS_FINAL7\yolov8n_train
--- Training for method 'train' completed. Re